<a href="https://colab.research.google.com/github/PrajwalRaj21/python/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch torchvision
!pip install transformers
!pip install opencv-python
!pip install google-colab


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [28]:
import os

# Verify the dataset path
dataset_path = '/content/drive/My Drive/Project Dataset/'

# Check if the path exists and list its contents
if os.path.exists(dataset_path):
    print("Dataset path exists!")
    print("Contents of the dataset directory:")
    print(os.listdir(dataset_path))
else:
    print("Dataset path does not exist. Please check the path.")


Dataset path exists!
Contents of the dataset directory:
['ख\xa0.png', 'क.jpg', 'ग\xa0.jpg', 'भाद्र.jpg', 'असोज\xa0.jpg', 'जेष्ठ.jpg']


In [33]:
import os

# Check if the dataset path exists
if not os.path.exists(dataset_path):
    raise ValueError("The dataset path does not exist.")

# List contents of the dataset path
dataset_contents = os.listdir(dataset_path)
print("Contents of the dataset path:")
for file_name in dataset_contents:
    print(file_name.encode('utf-8'))  # Print encoded file names

# Check if the dataset path contains any image files
image_files = [file for file in dataset_contents if file.endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff'))]
if not image_files:
    raise ValueError("The dataset path does not contain any image files.")


Contents of the dataset path:
b'\xe0\xa4\x96\xc2\xa0.png'
b'\xe0\xa4\x95.jpg'
b'\xe0\xa4\x97\xc2\xa0.jpg'
b'\xe0\xa4\xad\xe0\xa4\xbe\xe0\xa4\xa6\xe0\xa5\x8d\xe0\xa4\xb0.jpg'
b'\xe0\xa4\x85\xe0\xa4\xb8\xe0\xa5\x8b\xe0\xa4\x9c\xc2\xa0.jpg'
b'\xe0\xa4\x9c\xe0\xa5\x87\xe0\xa4\xb7\xe0\xa5\x8d\xe0\xa4\xa0.jpg'


In [38]:
import os

# Check if the dataset path exists
if not os.path.exists(dataset_path):
    raise ValueError("The dataset path does not exist.")

# List contents of the dataset path
dataset_contents = os.listdir(dataset_path)
print("Contents of the dataset path:")
for file_name in dataset_contents:
    print(file_name)  # Print file names

# Filter out non-image files and extract labels
image_files = []
labels = []
for file_name in dataset_contents:
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')):
        image_files.append(os.path.join(dataset_path, file_name))
        label = os.path.splitext(file_name)[0]  # Extract label from filename
        labels.append(label)

if not image_files:
    raise ValueError("The dataset path does not contain any image files.")

# Now 'image_files' contains paths to the image files and 'labels' contains the corresponding labels.


Contents of the dataset path:
ख .png
क.jpg
ग .jpg
भाद्र.jpg
असोज .jpg
जेष्ठ.jpg


In [44]:
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import torch

class SignLanguageDataset(Dataset):
    def __init__(self, dataset_path, transform=None):
        self.dataset_path = dataset_path
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.label_map = {}

        # Supported image file extensions
        valid_image_extensions = [".jpg", ".jpeg", ".png", ".bmp", ".gif", ".tiff"]

        # List contents of the dataset path
        dataset_contents = os.listdir(dataset_path)
        print("Contents of the dataset path:")
        for file_name in dataset_contents:
            print(file_name)  # Print file names

        # Filter out non-image files and extract labels
        for file_name in dataset_contents:
            if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')):
                image_path = os.path.join(dataset_path, file_name)
                self.image_paths.append(image_path)
                label = os.path.splitext(file_name)[0]  # Extract label from filename
                self.labels.append(label)
                if label not in self.label_map:
                    self.label_map[label] = len(self.label_map)

        if not self.image_paths:
            raise ValueError("The dataset path does not contain any image files.")

        self.num_classes = len(self.label_map)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(self.label_map[label])

# Define the transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Initialize the dataset
dataset_path = '/content/drive/My Drive/Project Dataset/'
dataset = SignLanguageDataset(dataset_path, transform=transform)

# Debug prints
print(f'Total dataset size: {len(dataset)}')

if len(dataset) == 0:
    raise ValueError("Dataset is empty. Please check the dataset path and ensure it contains image files.")

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
print(f'Train size: {train_size}, Validation size: {val_size}')

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


Contents of the dataset path:
ख .png
क.jpg
ग .jpg
भाद्र.jpg
असोज .jpg
जेष्ठ.jpg
Total dataset size: 6
Train size: 4, Validation size: 2


**Data Loader**

In [41]:
from torch.utils.data import DataLoader, random_split

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)


# **Loading and Training the ViT Model**

In [52]:
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from torch import nn, optim
from tqdm.notebook import tqdm

# Load feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=dataset.num_classes, ignore_mismatched_sizes=True)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {train_loss/len(train_loader)}')

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    print(f'Validation Loss: {val_loss/len(val_loader)}')


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/5, Loss: 1.9795479774475098
Validation Loss: 2.3795900344848633


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2/5, Loss: 1.0457419157028198
Validation Loss: 2.737720012664795


...

Epoch 3/5, Loss: 0.6030136346817017
Validation Loss: 3.019587516784668


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 4/5, Loss: 0.35102540254592896
Validation Loss: 3.232095718383789


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 5/5, Loss: 0.2011757493019104
Validation Loss: 3.404489517211914


# **Implementing the T5 Transformer**

In [54]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load pre-trained T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')
t5_model = T5ForConditionalGeneration.from_pretrained('t5-base')

def generate_text_from_gestures(gestures):
    input_text = " ".join(gestures)
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    output_ids = t5_model.generate(input_ids)
    text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return text


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **Camera Feed**

In [53]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

# JavaScript code to capture the camera feed
def start_camera():
    js = '''
    async function startCamera() {
        const video = document.createElement('video');
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        document.body.appendChild(video);
        await video.play();

        // Capture and send frames to Python
        setInterval(() => {
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            const frame = canvas.toDataURL('image/jpeg', 0.8);
            google.colab.kernel.invokeFunction('process_frame', [frame], {});
        }, 100);
    }
    startCamera();
    '''
    display(Javascript(js))

# Python function to process frames
def process_frame(frame):
    # Process the frame (e.g., convert to OpenCV format)
    frame_data = b64decode(frame.split(',')[1])
    nparr = np.frombuffer(frame_data, np.uint8)
    frame_cv2 = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    # Display the processed frame
    cv2.imshow('Camera Feed', frame_cv2)
    cv2.waitKey(1)

# Start the camera and process frames
start_camera()


<IPython.core.display.Javascript object>